# RF based on 500 selected SNPs from positive tail of distribution of rel. fitness betas


## Import packages

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 999)

import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Settings for seaborn
sns.set(color_codes=True)
sns.set(rc={'figure.figsize':(11.7,8.27)})


## Select specific SNPs
Select 500 SNPs from positive tail of selection coefficient distribution:

In [ ]:
betas = pd.read_csv('/Carnegie/DPB/Data/Shared/Labs/Moi/Everyone/deepselection/randomForest/betas_woNAs_55climvars_rFit.txt', sep='\t')
betas.rename(columns={'clim-bio18.assoc_y':'clim-bio18'}, inplace=True)
betas.drop(['clim-bio18.assoc_x'],axis=1, inplace=True)

In [ ]:
# Extract mlp and mli datasets
MLP = betas[['rs', 'rFitness2_mlp']]
MLI = betas[['rs', 'rFitness2_mli']]
THP = betas[['rs', 'rFitness2_thp']]
THI = betas[['rs', 'rFitness2_thi']]
THI

In [ ]:
# check distribution of betas from GWAS to rel. Fitness

a = MLP['rFitness2_mlp']
b = MLI['rFitness2_mli']
c = THP['rFitness2_thp']
d = THI['rFitness2_thi']

sns.set()
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize = (24, 6))
fig.suptitle('Distribution of betas from GWAS to fitness') 
sns.distplot(a, ax=ax1)
sns.distplot(b, ax=ax2)
sns.distplot(c, ax=ax3)
sns.distplot(d, ax=ax4)
ax1.set_xlabel('MLP')
ax2.set_xlabel('MLI')
ax3.set_xlabel('THP')
ax4.set_xlabel('THI')
fig.show()
#fig.savefig('Output/03_BetasDist.png', bbox_inches='tight', dpi=600)

In [ ]:
# Sort & select
MLP = MLP.sort_values(by=['rFitness2_mlp'], ascending=False)
MLI = MLI.sort_values(by=['rFitness2_mli'], ascending=False)
THP = THP.sort_values(by=['rFitness2_thp'], ascending=False)
THI = THI.sort_values(by=['rFitness2_thi'], ascending=False)

# get the first and last 1000 objects (highest and lowest betas)
selMLP = MLP.iloc[:500, :]   
#selMLP = selMLP.append(MLP.iloc[-500:, :])
selMLPSNPs = selMLP['rs'].tolist()

selMLI = MLI.iloc[:500, :]   
#selMLI = selMLI.append(MLI.iloc[-500:, :])
selMLISNPs = selMLI['rs'].tolist()

selTHP = THP.iloc[:500, :]   
#selTHP = selTHP.append(THP.iloc[-500:, :])
selTHPSNPs = selTHP['rs'].tolist()

selTHI = THI.iloc[:500, :]   
#selTHI = selTHI.append(THI.iloc[-500:, :])
selTHISNPs = selTHI['rs'].tolist()

In [ ]:
selTHI

In [ ]:
# check distribution of selection

a = selMLP['rFitness2_mlp']
b = selMLI['rFitness2_mli']
c = selTHP['rFitness2_thp']
d = selTHI['rFitness2_thi']

sns.set()
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize = (24, 6))
fig.suptitle('Distribution of selected betas from GWAS to fitness') 
sns.distplot(a, ax=ax1)
sns.distplot(b, ax=ax2)
sns.distplot(c, ax=ax3)
sns.distplot(d, ax=ax4)
ax1.set_xlabel('MLP')
ax2.set_xlabel('MLI')
ax3.set_xlabel('THP')
ax4.set_xlabel('THI')
fig.show()
#fig.savefig('Output/03_SelBetasDist.png', bbox_inches='tight', dpi=600)

In [ ]:
# Use all rs from the selection above and create new list
mySNPs = selMLPSNPs + selMLISNPs + selTHPSNPs + selTHISNPs
len(mySNPs)

In [ ]:
# check for duplicates
from collections import Counter
[k for k,v in Counter(mySNPs).items() if v>1]
len(mySNPs)

In [ ]:
# remove duplicates
mySNPs = list(set(mySNPs))

# check again for duplicates
from collections import Counter
[k for k,v in Counter(mySNPs).items() if v>1]
len(mySNPs)

In [ ]:
# create now target dataframe with selected SNPs

target = pd.DataFrame(mySNPs, columns=['rs'])

a = target.join(MLP.set_index('rs'), on='rs')
a.rename(columns={'rFitness2_mlp':'rFitness'}, inplace=True)
a['locat'] = 'MLP'

b = target.join(MLI.set_index('rs'), on='rs')
b.rename(columns={'rFitness2_mli':'rFitness'}, inplace=True)
b['locat'] = 'MLI'

c = target.join(THP.set_index('rs'), on='rs')
c.rename(columns={'rFitness2_thp':'rFitness'}, inplace=True)
c['locat'] = 'THP'

d = target.join(THI.set_index('rs'), on='rs')
d.rename(columns={'rFitness2_thi':'rFitness'}, inplace=True)
d['locat'] = 'THI'

target = a.append([b, c, d], ignore_index=True, sort=False)
target = target.reset_index()
target


In [ ]:
tarMLP = target[target["locat"] == 'MLP']
tarMLP #index0-1975


In [ ]:
tarMLI = target[target["locat"] == 'MLI']
tarMLI #index 1976-3951

In [ ]:
tarTHP = target[target["locat"] == 'THP']
tarTHP #index 3952-5927

In [ ]:
tarTHI = target[target["locat"] == 'THI']
tarTHI #index 5928-7903

In [ ]:
# check distribution of target
sns.distplot(target['rFitness'])
plt.xlabel('Combined beta values as target variable')
plt.title('Distribution of target')
#plt.savefig('Output/03_TargetVarDist.png', bbox_inches='tight', dpi=600)

In [ ]:
predictors = pd.DataFrame(mySNPs, columns=['rs'])
predictors = predictors.join(betas.set_index('rs'), on='rs')
cols=[1,2,3,4]    #drop rFitness columns
predictors = predictors.drop(predictors.columns[cols], axis=1)
predictors = pd.concat([predictors]*4, ignore_index=True)
predictors

In [ ]:
# add annotation to predictors dataset
annot = pd.read_csv(f'/Carnegie/DPB/Data/Shared/Labs/Moi/Everyone/deepselection/randomForest/515g2.ann.txt', sep='\t')
predictors = predictors.join(annot.set_index('rs'), on='rs')
predictors = predictors.drop(columns=['chr', 'ps', 'allel1', 'allel2'])
predictors

In [ ]:
len(predictors)/2

In [ ]:
# encode annotation numerically
lb = LabelEncoder()
predictors['ann'] = lb.fit_transform(predictors['ann'])

# print encoding
lbMapping = dict(zip(lb.classes_, lb.transform(lb.classes_)))
lbMapping

In [ ]:
# prepare climate data 

clim = pd.read_csv(f'/Carnegie/DPB/Data/Shared/Labs/Moi/Everyone/natvar/climate/2029gclimate.tsv', delim_whitespace=True)
climT = pd.concat([clim.iloc[[1813]]]*int(len(predictors)/2), ignore_index=True) #1813 = accession close to Tübingen
climM = pd.concat([clim.iloc[[1845]]]*int(len(predictors)/2), ignore_index=True) #1845 = accession close to Madrid
climFin = pd.concat([climM, climT], axis=0) #concat this way, to have Madrid at first, then Tübingen
climFin = climFin.iloc[:, :-12]    
climFin


In [ ]:
# finalize predictors dataset
predictors = pd.concat([predictors.reset_index(drop=True), climFin.reset_index(drop=True)], axis=1, sort=False)  # without reset_index, NAs were introduced in DF
predictors

In [ ]:
# check distribution of predictors

a = predictors['ann']
b = predictors['bio1']
c = predictors['clim-bio1']

sns.set()
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (24, 6))
fig.suptitle('Distribution of beta values from selected predictor variables') 
sns.distplot(a, ax=ax1)
sns.distplot(b, ax=ax2)
sns.distplot(c, ax=ax3)
ax1.set_xlabel('Annotation')
ax2.set_xlabel('Worldclim | bio1')
ax3.set_xlabel('Betas | bio1')
fig.show()
#ig.savefig('Output/03_PredVarDist.png', bbox_inches='tight', dpi=600)


## Random Forest
### Input variable preparation and distribution plots

In [ ]:
y = target['rFitness']
X = predictors.iloc[:, 1:].copy()    # without rs column

### Packages

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from sklearn import metrics
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from yellowbrick.regressor import PredictionError, ResidualsPlot
from yellowbrick.features import Rank1D

***

In [ ]:
# Fit regression model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
regr_rf = RandomForestRegressor(oob_score=True, random_state=0, n_estimators = 500)

In [ ]:
y_test

In [ ]:
y_test2plot = y_test.copy()
y_test2plot = y_test2plot.reset_index()
y_test2plot['locat'] = ''
y_test2plot

In [ ]:
y_test2plot['locat'] = ['MLP' if 0 <= x <= 1975 else 'MLI' if 1976 <= x <= 3951 else 'THP' if 3952 <= x <= 5927 else 'THI' for x in y_test2plot['index']]

In [ ]:
y_test2plot

In [ ]:
# Training
regr_rf.fit(X_train, y_train)

In [ ]:
# Prediction
y_rf = regr_rf.predict(X_test)

In [ ]:
y_rf

In [ ]:
predicted_train = regr_rf.predict(X_train)
predicted_test = regr_rf.predict(X_test)
test_score = r2_score(y_test, predicted_test)
spearman = spearmanr(y_test, predicted_test)
pearson = pearsonr(y_test, predicted_test)

In [ ]:
# Metrics --> printed to file
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_rf), file=open('Output/03_Metrics.txt', 'a'))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_rf), file=open('Output/03_Metrics.txt', 'a'))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_rf)), file=open('Output/03_Metrics.txt', 'a'))

print('Further statistics:', file=open('Output/03_Metrics.txt', 'a'))
print(f'Out-of-bag R2 score estimate: {regr_rf.oob_score_:>5.3}', file=open('Output/03_Metrics.txt', 'a'))
print(f'Test data R2 score: {test_score:>5.3}', file=open('Output/03_Metrics.txt', 'a'))
print(f'Test data Spearman correlation: {spearman[0]:.3}', file=open('Output/03_Metrics.txt', 'a'))
print(f'Test data Pearson correlation: {pearson[0]:.3}', file=open('Output/03_Metrics.txt', 'a'))

#### Plot results with Yellowbrick
https://www.scikit-yb.org/en/latest/api/regressor/peplot.html

In [ ]:
# Residuals plot
f = plt.figure()
visualizer = ResidualsPlot(regr_rf)
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.show()
#f.savefig("Output/03_Residuals.png", bbox_inches='tight', dpi=600)

In [ ]:
# Prediction error plot
f = plt.figure()
visualizer = PredictionError(regr_rf)
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.show()
#f.savefig("Output/03_PredActual.png", bbox_inches='tight', dpi=600)

In [ ]:
y_rf2plot = pd.DataFrame(y_rf)
df2plot = pd.concat([y_test2plot, y_rf2plot], axis=1)
df2plot.columns = ['index', 'actual', 'location', 'pred']
df2plot

In [ ]:
sns.set_palette('Paired')
sns.scatterplot(x='actual', y='pred', hue='location', data=df2plot)
plt.title("Actual vs predicted beta values")
plt.xlabel("Actual")
plt.ylabel("Predicted")
#lt.savefig('Output/03_PredActual_Color.png', bbox_inches='tight', dpi=600)

Feature importance:
https://machinelearningmastery.com/calculate-feature-importance-with-python/

In [ ]:
importance = regr_rf.feature_importances_         # get importance

# summarize feature importance
#for i,v in enumerate(importance):
#	print('Feature: %0d, Score: %.5f' % (i,v))

labels = list(X.columns.values)

plt.figure(figsize=(18,8))
imp = sns.barplot([x for x in range(len(importance))], importance)
imp.set_xticklabels(labels,  rotation='vertical')
plt.title("Feature importance")
#plt.savefig('Output/03_Features.png', bbox_inches='tight', dpi=600)
plt.show()